# Guía de ciencia de datos y machine learning

Este documento tiene como finalidad resumir los métodos más cómunes empleados en la ciencia de datos y machine learning.

- ## Carga, limpieza y transformación de los datos

#### Importando librerías para la limpieza, carga y visualización de los datos

In [ ]:
import pandas as pd
import numpy as p
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 

warnings.filterwarnings('ignore') # ignora las advertencias
pd.set_option('display.max_columns', None) # visualiza todas las columnas
pd.options.display.float_format = '{:.2f}'.format # los datos numericos contendran 2 decimales

#### Cargando datos desde archivos

In [ ]:
data = pd.read_csv('archivo.csv') # cargando desde archivo csv
data = pd.read_excel('archivo.xlsx') # cargando desde archivo excel
data = pd.read_csv('dDEC1204.txt', sep=',', header=None, encoding='ISO-8859-1') # cargando desde archivo txt

#### Cargando datos desde base de datos

In [ ]:
import mysql.connector

conexion = mysql.connector.connect(host='localhost', user='root', password='password', database='database') # conexión a la BD
data_sql = pd.read_sql('SELECT * FROM tareas', conexion) # ejecución de la consulta SQL
conexion.close() # cerrando conexión a la BD
data = pd.DataFrame(data_sql) # guardando la consulta en un dataFrame

#### Visualizar información del dataframe

In [ ]:
data.info()

#### Visualizando la estructura del dataframe

In [ ]:
print(f'El dataframe contiene {data.shape[0]} filas y {data.shape[1]} columnas')

### Limpieza y preparación de datos

#### Eliminando columnas innecesarias

In [ ]:
data.drop(columns=['id', 'fecha'], inplace=True) # elimina las columnas id y fecha

#### Seleccionando columnas a usar

In [ ]:
data= data[['Agente', 'CENTRAL', 'Tipo de central']] # selecciona unicamente esas 3 columnas

#### Eliminando filas duplicadas

In [ ]:
data.drop_duplicates(inplace=True) # elimina las filas duplicadas

#### Visualizando columnas con valores nulos

In [ ]:
null = data.isnull().sum() * 100 / data.shape[0]
null[null > 0].sort_values(ascending=False) # verifica los nulos de las columnas

#### Eliminando filas con valores nulos en una columna

In [ ]:
data.dropna(subset=['categoria'], inplace=True) # elimina las filas nulas de categoria

#### Rellenando valores faltantes en una columna

In [ ]:
data['distancia'].fillna(0, inplace=True) # rellanando con 0
data['distancia'].fillna(data['distancia'].mean(), inplace=True) # rellanando con la media
data['distancia'].fillna(data['distancia'].mode()[0]), inplace=True) # rellanando con la moda
data['distancia'].fillna(data['distancia'].median(), inplace=True) # rellanando con la mediana

#### Reemplazando valores en una columna

In [ ]:
data['reviews'].replace(np.nan, 0, inplace=True) # reemplaza los nan por 0

#### Eliminando filas con un valor específico

In [ ]:
data = data[data['categoria'] != 'desconocido'] # elimina las filas que contengan desconocido

#### Renombrando columnas

In [ ]:
data.rename(columns={0: 'Central', 'tienda': 'lugar'}, inplace=True) # renombra 0 y tienda

#### Creando una columna nueva

In [ ]:
data['price_night'] = data['price'] / data['minimum_nights'] # price x minimum_nights

#### Filtrando dataframe

In [ ]:
# Filtrando por agente EMGESA S.A. y Tipo de central H o T
data = data[(data['Agente'] == 'EMGESA S.A.') & (data['Tipo de central'].isin(['H', 'T']))]

#### Combinando dataframes

In [ ]:
# creando dataframe usando data_1 y data_2
data = pd.merge(data_1, data_2, left_on='CENTRAL', right_on='Central', how='inner')

#### Cambiando tipo de datos de una columna

In [ ]:
data['codigo_categoria'] = data['codigo_categoria'].astype(str) # cambia a string
data['fecha_venta'] = pd.to_datetime(data['fecha_venta']) # cambia a datetime
data['fecha'] = data['fecha_venta'].dt.date # crea columna date
data['hora'] = data['fecha_venta'].dt.time # crea columna time
data['hora'] = pd.to_datetime(data['hora'].apply(lambda x: x.strftime('%H:%M:%S'))) # cambia a formato HH:MM:SS

- ## Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

df = data.copy(deep = True) # creando copia del dataframe que se transformara

le = LabelEncoder() # creando label encoder
text_data_features = [i for i in list(data.columns) if i not in list(data.describe().columns)] # obteniendo variables cat

print('Label Encoder Transformation')
for i in text_data_features:
    df[i] = le.fit_transform(df[i])
    print(i, ' : ',df[i].unique(), ' = ', le.inverse_transform(df[i].unique()))

- ## Dividiendo datos categóricos y numéricos

In [ ]:
columns = list(df.columns)

categorical_features = []
numerical_features = []

for feature in columns:
    if data[feature].nunique() > 6:
        numerical_features.append(feature)
    else:
        categorical_features.append(feature)
        
print('Categorical Features:')
print(', '.join(categorical_features))
print('Numerical Features:')
print(', '.join(numerical_features))

Aquí, las características categóricas se definen si el atributo tiene menos de 6 elementos únicos, de lo contrario, es una característica numérica. El enfoque típico para esta división de características también puede basarse en los tipos de datos de los elementos del atributo respectivo.

- ## Análisis Exploratorio de datos

Para el EDA se usará el dataframe data

#### Estadística descriptiva

In [ ]:
data.describe(include = 'O').T # variables categóricas
data.describe().T # variables numericas
round(data['venta'].describe().to_frame().T, 1) # una sola variable numerica
data['categoria'].describe(include='O').to_frame().T # una sola variable numerica

In [ ]:
# promedios de age, income y spending por categoria
df.groupby('categoria')[['Age', 'Annual Income (k$)', 'Spending Score (1-100)']].mean()

Acá se gráfica y se analizan los datos para luego proceder a los siguientes pasos.

Para las transformaciones y optimizaciones se usará el dataframe df.

- ## Estandarizado y normalizado de datos

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

MinMaxScaler = MinMaxScaler() #Para la normalizacion de los datos
StandarScaler = StandardScaler() #Para el escalado de los datos.

df['tenure'] = MinMaxScaler.fit_transform(df[['tenure']])
df['charges'] = scaler.fit_transform(df[['charges']])

In [ ]:
# creando MinMaxScaler
MinMaxScaler = MinMaxScaler()
# aplicando MinMaxScaler a todas las columnas numericas
df[df.columns] = MinMaxScaler.fit_transform(df[df.columns])

# creando StandardScaler
StandarScaler = StandardScaler()
# aplicando StandardScaler a todas las columnas numericas
df[df.columns] = StandarScaler.fit_transform(df[df.columns])

**Normalización:** Esta técnica se aplica a características cuyos datos no siguen una distribución normal (gaussiana). Reescala los valores de una característica a un rango entre 0 y 1. Cuando la distribución esta sesgada hacia un lado, se emplea normalización.

**Estandarización:** Este método se utiliza para características que siguen una distribución normal o tienen valores significativamente más grandes o más pequeños que los de otras características. La estandarización transforma los valores de una característica para que tengan una media de 0 y una desviación estándar de 1.

- ## Análisis de correlación

#### Mapa de calor

In [ ]:
plt.figure(figsize = (20,5))
sns.heatmap(df.corr(),cmap= colors, annot = True)
plt.show()

#### Correlación con respecto a la variable objetivo

In [ ]:
Matrix_Corr = df.corrwith(df['Churn']).sort_values(ascending = False).to_frame()
Matrix_Corr.columns = ['Correlations']
plt.subplots(figsize = (5,5))
sns.heatmap(Matrix_Corr,annot = True,cmap = colors,linewidths = 0.4,linecolor = 'black')
plt.title('Correlation Outcome')

Eliminamos las características con un coeficiente de correlación entre (-0.1, 0.1).

- ##  Selección de Variables categóricas

#### Test cuadrados chi

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, mutual_info_classif 

In [ ]:
features = df.loc[:,categorical_features]
target = df.loc[:,'Churn']

best_features = SelectKBest(score_func = chi2,k = 'all')
fit = best_features.fit(features,target)

featureScores = pd.DataFrame(data = fit.scores_,index = list(features.columns),columns = ['Chi Squared Score']) 

plt.subplots(figsize = (5,5))
sns.heatmap(featureScores.sort_values(ascending = False,by = 'Chi Squared Score'),annot = True,cmap = colors,linewidths = 0.4,linecolor = 'black',fmt = '.2f')
plt.title('Selection of Categorical Features')

Se eliminan las columnas que tengan muy poca correlación con la columna objetivo.

- ##  Selección de Variables númericas

#### Test de ANNOVA

In [ ]:
from sklearn.feature_selection import f_classif

In [ ]:
features = df.loc[:,numerical_features]
target = df.loc[:,'Churn']

best_features = SelectKBest(score_func = f_classif,k = 'all')
fit = best_features.fit(features,target)

featureScores = pd.DataFrame(data = fit.scores_,index = list(features.columns),columns = ['ANOVA Score']) 

plt.subplots(figsize = (5,5))
sns.heatmap(featureScores.sort_values(ascending = False,by = 'ANOVA Score'),annot = True,cmap = colors,linewidths = 0.4,linecolor = 'black',fmt = '.2f')
plt.title('Selection of Numerical Features')

Se eliminan las columnas que tengan muy poca correlación con la columna objetivo.

- ## Train Test Split

Es una técnica utilizada en machine learning para dividir un conjunto de datos en dos partes: un conjunto de entrenamiento y un conjunto de prueba. Esta división es fundamental para evaluar el rendimiento de un modelo de machine learning de manera imparcial.

#### Selección de variable objetivo (y) y variables independientes (X)

In [ ]:
X = df.drop(columns='stroke')    
y = df['stroke']

- ## Balance entre clases por sobremuestreo (SMOTE)

El desbalance entre clases puede repercutir negativamente en el entrenamiento y aplicación del modelo. Para solucionar este problema se debe realizar un balance entre clases por sobremuestreo u alguno otra técnica.

SMOTE genera muestras sintéticas para la clase minoritaria con el objetivo de equilibrar la distribución de clases. Creando nuevos ejemplos sintéticos mediante la interpolación de características de ejemplos vecinos.

#### Visualizando clases

In [ ]:
c = sns.countplot(data=df, x='stroke', palette='pastel')
c.set(title='Cantidad de pacientes ACV+/ACV-', ylabel ='Número de pacientes', xlabel='')
c.set_xticklabels(['ACV-', 'ACV+'])
plt.show()

#### Creando muestras sintéticas con SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

cs = sns.countplot(x=y_resampled, palette='pastel')
cs.set(title='Cantidad de pacientes ACV+/ACV-', ylabel ='Número de pacientes', xlabel='')
cs.set_xticklabels(['ACV-', 'ACV+'])

- ## Datos de prueba y datos de entrenamiento

In [ ]:
from sklearn.model_selection import train_test_split
# Defino datos de entrenamiento (70%) y datos de prueba(30%)
# Random_state:Establece una semilla de múmeros aleatorios que asegura que el código sea reproducible cada vez que lo corra
# shuffle=True: Mezcla el orden de los datos para evitar sesgos
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42, shuffle=True)

- ## Manejo de valores atípicos

#### Método Z-Score

In [ ]:
from scipy import stats
import math

# seleccionando columnas a las que se les eliminaran los outliers
selected_columns = ['Credit_Limit', 'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1',
                    'Total_Trans_Amt', 'Total_Ct_Chng_Q4_Q1']

# calcular Z-scores para las columnas seleccionadas en la training data
z_scores = np.abs(stats.zscore(X_train[selected_columns]))

# establecer valor de threshold para detectar outliers
threshold = 3

# encontrar el indice de los outliers basado en el threshold
outlier_indices = np.where(z_scores > threshold)[0]

# remover los outliers de la training data
X_train = X_train.drop(X_train.index[outlier_indices])
y_train = y_train.drop(y_train.index[outlier_indices])

- ## Modelos predictivos

Acá procederemos a desarrollar los algoritmos para problemas de clasificación, regresión y clustering.

- ## Clasificación

#### Random Forest Classifier

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# creando Random Forest Classifier
rfc = RandomForestClassifier()

# Definiendo los hiperparámetros 
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'random_state': [0, 42]
}
# desarrollando un grid search con validación cruzada para encontrar los mejores hiperparametros
grid_search = GridSearchCV(rfc, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# imprimiendo los mejores hiperparametros
print(grid_search.best_params_)

In [ ]:
# entrenando el modelo
rfc = RandomForestClassifier(classifier__max_depth: 10, classifier__min_samples_leaf: 4,
                             classifier__min_samples_split: 10, classifier__n_estimators: 50, random_state: 42)
rfc.fit(X_train, y_train)

In [ ]:
# realizando las predicciones
rfc_pred = rfc.predict(X_test)

#### Decision Tree Classifier

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

# creando Decision Tree Classifier
dtree = DecisionTreeClassifier()

# Definiendo los hiperparámetros 
param_grid = {
    'max_depth': [2, 4, 6, 8],
    'min_samples_split': [2, 4, 6, 8],
    'min_samples_leaf': [1, 2, 3, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'random_state': [0, 42]
}
# desarrollando un grid search con validación cruzada para encontrar los mejores hiperparametros
grid_search = GridSearchCV(dtree, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# imprimiendo los mejores hiperparametros
print(grid_search.best_params_)

In [ ]:
# entrenando el modelo
dtree = DecisionTreeClassifier(random_state=42, max_depth=2, max_features='sqrt', min_samples_leaf=4, min_samples_split=2)
dtree.fit(X_train, y_train)

In [ ]:
# creando las predicciones
dtree_pred = dtree.predict(X_test)

#### XGBoost Classifier

In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

# creando XGBoost classifier
xgbc = XGBClassifier()

# definiendo hiperparametros para el XGBoost
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'subsample': [0.8, 1.0],
    'random_state': [0, 42]
}
# desarrollando un grid search con validación cruzada para encontrar los mejores hiperparametros
grid_search = GridSearchCV(xgbc, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# imprimiendo mejores hiperparametros
print(grid_search.best_params_)

In [ ]:
# entrenando el modelo
xgbc = XGBClassifier(max_depth=3, learning_rate=0.1, n_estimators=100, random_state=0, subsample=0.8)
xgbc.fit(X_train, y_train)

In [ ]:
# creando predicciones
xgbc_pred = xgbc.predict(X_test)

#### k-Nearest Neightbors (k-NN)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

# creando modelo knn
knn = KNeighborsClassifier()

# definiendo hiperparametros para el knn
param_grid = {
    'n_neighbors': [3, 5, 7, 9],  # Número de vecinos
    'weights': ['uniform', 'distance'],  # Método de ponderación de los vecinos
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  # Algoritmo para calcular los vecinos
    'p': [1, 2],  # Distancia de Minkowski (1 para Manhattan, 2 para Euclidiana)
    'random_state': [0, 42]
}
# desarrollando un grid search con validación cruzada para encontrar los mejores hiperparametros
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X, y)

# imprimiendo mejores hiperparametros
print(grid_search.best_params_)

In [ ]:
# entrenando el modelo
knn = KNeighborsClassifier(n_neighbors=3, weights='uniform', algorithm='auto', p=2, random_state=42)
knn.fit(X_train, y_train)

In [ ]:
# creando predicciones
knn_pred = knn.predict(X_test)

#### SVC (Support Vector Classifier)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# creando modelo SVM
svc = SVC()

# parametros para el grid search
param_grid = {'C': [0.1, 1, 10, 100],
              'degree': [2, 3, 4, 5],
              'gamma': ['scale', 'auto'],
              'random_state': [0,42]
}
# desarrollando un grid search con validación cruzada para encontrar los mejores hiperparametros
grid = GridSearchCV(svc, param_grid, refit=True, verbose=3, cv=5, n_jobs=-1)
grid.fit(X_train,y_train)

# imprimiendo mejores parametros
print(grid.best_params_)

In [ ]:
# entrenando el modelo
svc = SVC(C=0.1, degree=2, gamma='auto', random_state=0, kernel='linear')
svc.fit(X_train,y_train)

In [ ]:
# creando predicciones
svc_pred = svc.predict(X_test)

- ## Evaluación del modelo - Clasificación

#### Classification report

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

print("Model Accuracy: ", accuracy_score(y_test, knn_pred))
print(classification_report(y_test, knn_pred))

**Precisión:** La precisión indica la proporción de predicciones positivas correctas respecto al total de predicciones positivas realizadas por el modelo.

**Recall:** El recall indica la proporción de instancias positivas reales que fueron correctamente identificadas por el modelo.

**F1-Score:** La puntuación F1 es la media armónica de precisión y recall. Proporciona una medida de precisión equilibrada con el recall.

**Support:** El soporte indica el número de instancias reales para cada clase en los datos de prueba.

**Accuracy:** La exactitud del modelo indica la proporción de predicciones correctas respecto al total de predicciones realizadas por el modelo.

**Macro Avg:** El promedio de precisión, recall o F1-score sin considerar la proporción para cada clase.

**Weighted Avg:** El promedio de precisión, recall o F1-score considerando la proporción para cada clase.

#### Confusion Matriz

El modelo entrenado se evalúa en el conjunto de prueba. La matriz de confusión se utiliza para visualizar el rendimiento del modelo. Muestra las predicciones de verdaderos positivos, verdaderos negativos, falsos positivos y falsos negativos del modelo.

Precision: La precisión es una medida de cuántas de las predicciones verdaderamente positivas fueron realmente correctas. Se define como el número de verdaderos positivos (TP) dividido por la suma de verdaderos positivos (TP) y falsos positivos (FP).

Recall: La recuperación (o Sensibilidad) es una medida de cuántos de los casos positivos reales fueron identificados correctamente. Se define como el número de verdaderos positivos (TP) dividido por la suma de verdaderos positivos (TP) y falsos negativos (FN).

F1-Score: La puntuación F1 es la media armónica de Precisión y Recuperación e intenta encontrar el equilibrio entre precisión y recuperación. Se define como 2 veces el producto de precisión y recuperación dividido por la suma de precisión y recuperación.

In [ ]:
from sklearn.metrics import confusion_matrix

# matriz de confusion
cm = confusion_matrix(y_test, knn_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

array([[900, 136],
       [133, 240]])

**True Negatives (TN): 900** - El modelo predijo correctamente la clase 0 (no churn) para estas instancias.

**False Positives (FP): 136** - El modelo predijo incorrectamente la clase 1 (churn) para estas instancias, que en realidad son de la clase 0 (no churn).

**False Negatives (FN): 133** - El modelo predijo incorrectamente la clase 0 (no churn) para estas instancias, que en realidad son de la clase 1 (churn).

**True Positives (TP): 240** - El modelo predijo correctamente la clase 1 (churn) para estas instancias.

#### Importancia de las características

In [ ]:
imp_df = pd.DataFrame({
    "Feature Name": X_train.columns,
    "Importance": dtree.feature_importances_
})
fi = imp_df.sort_values(by="Importance", ascending=False)

fi2 = fi.head(10)
plt.figure(figsize=(10,4))
sns.barplot(data=fi2, x='Importance', y='Feature Name')
plt.title('Importancia de caracteristica (Decision Tree Regressor)', fontsize=18)
plt.xlabel ('Importancia', fontsize=16)
plt.ylabel ('Caracteristica', fontsize=16)
plt.show()

#### Curva ROC y AUC

La curva ROC (Receiver Operating Characteristic) es una herramienta utilizada para evaluar el rendimiento de un modelo de clasificación binaria en función de su capacidad para discriminar entre clases positivas y negativas. La curva ROC representa la tasa de verdaderos positivos (sensibilidad) en el eje y frente a la tasa de falsos positivos (1 - especificidad) en el eje x para diferentes umbrales de clasificación.

La curva ROC es útil porque proporciona una representación gráfica de la capacidad de discriminación de un modelo en diferentes niveles de sensibilidad y especificidad. Además, el área bajo la curva ROC (AUC-ROC) es una métrica comúnmente utilizada para cuantificar la capacidad discriminativa global del modelo. Un AUC-ROC cercano a 1 indica un modelo excelente que puede distinguir perfectamente entre clases positivas y negativas, mientras que un valor cercano a 0.5 sugiere que el modelo es similar a una clasificación aleatoria.

In [ ]:
from sklearn.metrics import roc_auc_score

# Calcula las probabilidades de predicción para la clase positiva
y_score = dtree.predict_proba(X_test)[:, 1]

# Calcula la tasa de verdaderos positivos y la tasa de falsos positivos
fpr, tpr, _ = roc_curve(y_test, y_score)

# Calcula el área bajo la curva ROC (AUC)
roc_auc = auc(fpr, tpr)

# Grafica la curva ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

#### Tabla resumen

| Modelo                |        Precision      | Recall                |F1-score              | Falsos negativos|
|-----------------------|-----------------------|-----------------------|-----------------------|-----------|
| Regresión logistica        | 0.96          | 0.96                     |0.96                   |118|       
| k-NN                       |  0.91         | 0.90                     |0.90                   |20|        
| Regresión logistica + PCA  |  0.94         |0.94                      |0.94                   |70 |
| k-NN + PCA                 | 0.81         | 0.81                     |0.81                    |135| 

- ## Regresión

#### XGBoost Regressor

In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

# creando XGBoost Regressor
xgbr = XGBRegressor()

# definiendo hiperparametros para el XGBoost
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'subsample': [0.8, 1.0],
    'random_state': [0, 42]
}
# desarrollando un grid search con validación cruzada para encontrar los mejores hiperparametros
grid_search = GridSearchCV(xgbr, param_grid, cv=5, scoring='r2')
grid_search.fit(X_train, y_train)

# imprimiendo mejores hiperparametros
print(grid.best_params_)

In [ ]:
# entrenando el modelo
xgbr = XGBRegressor(max_depth=7, learning_rate=0.1, n_estimators=300, random_state=42, subsample=0.8)
xgbr.fit(X_train, y_train)

In [ ]:
# creando predicciones
xgbr_pred = xgbr.predict(X_test)

#### Decision Tree Regressor

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

# creando Decision Tree Regressor
dtree = DecisionTreeRegressor()

# Definiendo los hiperparámetros a ajustar y sus valores
param_grid = {
    'max_depth': [2, 4, 6, 8],
    'min_samples_split': [2, 4, 6, 8],
    'min_samples_leaf': [1, 2, 3, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'random_state': [0, 42]
}
# desarrollando un grid search con validación cruzada para encontrar los mejores hiperparametros
grid_search = GridSearchCV(dtree, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# imprimiendo los mejores hiperparametros
print(grid_search.best_params_)

In [ ]:
# entrenando el modelo
dtree = DecisionTreeRegressor(random_state=0, max_depth=8, max_features='sqrt', min_samples_leaf=2, min_samples_split=6)
dtree.fit(X_train, y_train)

In [ ]:
# creando las predicciones
dtree_pred = dtree.predict(X_test)

- ## Evaluación del modelo - Regresión

#### Distribución

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(20,5))
# Regresión lineal
sns.distplot(y_test,ax=ax[0])
sns.distplot(rlp_pred,ax=ax[0])
# Ridge Regression
sns.distplot(y_test,ax=ax[1])
sns.distplot(rr_pred,ax=ax[1])
# Random Forest Regression
sns.distplot(y_test,ax=ax[2])
sns.distplot(rfr_pred,ax=ax[2])

# leyendas
ax[0].legend(["Precio actual", "Predicción"])
ax[1].legend(["Precio actual", "Predicción"])
ax[2].legend(["Precio actual", "Predicción"])

# titulos
ax[0].set_title("Regresión Lineal")
ax[1].set_title("Ridge Regression")
ax[2].set_title("Random Forest Regression")
plt.show()

#### Errores

In [ ]:
from sklearn import metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

fig, ax = plt.subplots(1,3,figsize=(20,5))
sns.barplot(x=["Regresión Lineal","Ridge Regression","Random Forest"], y=[mean_absolute_error(y_test, rlp_pred), mean_absolute_error(y_test, rr_pred), mean_absolute_error(y_test, rfr_pred)], ax=ax[0])
sns.barplot(x=["Regresión Lineal","Ridge Regression","Random Forest"], y=[mean_squared_error(y_test, rlp_pred), mean_squared_error(y_test, rr_pred), mean_squared_error(y_test, rfr_pred)], ax=ax[1])
sns.barplot(x=["Regresión Lineal","Ridge Regression","Random Forest"], y=[np.sqrt(mean_squared_error(y_test, rlp_pred)), np.sqrt(mean_squared_error(y_test, rr_pred)), np.sqrt(mean_squared_error(y_test, rfr_pred))], ax=ax[2])

ax[0].set_ylabel("Mean Absolute Error")
ax[1].set_ylabel("Mean Squared Error")
ax[2].set_ylabel("Root Mean Squared Error")
plt.show()

#### Accuracy

In [ ]:
from sklearn import metrics
from sklearn.metrics import r2_score

models = ["Linear Regression", "Ridge Regression", "Random Forest Regression"]
accuracy = [metrics.r2_score(y_test, rlp_pred),metrics.r2_score(y_test, rr_pred),metrics.r2_score(y_test, rfr_pred)]
plt.figure(figsize=(8,6))
sns.barplot(x=models, y=accuracy)

for i, val in enumerate(accuracy):
    plt.text(i, val, f'{val:.4f}', ha="center", va="bottom", fontsize=12)
    
plt.title("Comparación de Accuracy")
plt.ylabel("Accuracy")
plt.show()

#### Importancia de las características

In [ ]:
imp_df = pd.DataFrame({
    "Feature Name": X_train.columns,
    "Importance": dtree.feature_importances_
})
fi = imp_df.sort_values(by="Importance", ascending=False)

fi2 = fi.head(10)
plt.figure(figsize=(10,4))
sns.barplot(data=fi2, x='Importance', y='Feature Name')
plt.title('Importancia de caracteristica (Decision Tree Regressor)', fontsize=18)
plt.xlabel ('Importancia', fontsize=16)
plt.ylabel ('Caracteristica', fontsize=16)
plt.show()

- ## Clustering

In [ ]:
# creando variable que contiene los valores de las columnas Annual Income y Spending Score
X = df.iloc[:,[3,4]].values

In [ ]:
from sklearn.cluster import KMeans

# buscando el valor WCSS(Within Clusters Sum of Squares) usando distintos numeros de clusters
wcss = []

for i in range(1,11):
  kmeans = KMeans(n_clusters=i, init="k-means++", random_state=42)
  kmeans.fit(X)
  wcss.append(kmeans.inertia_)

In [ ]:
# grafico Elbow
plt.plot(range(1,11), wcss)
plt.title("Gráfico Elbow")
plt.xlabel("Número de Clústers")
plt.ylabel("WCSS")
plt.show()

In [ ]:
# definiendo el modelo
kmeans = KMeans(n_clusters=5, init='k-means++', random_state=0)

# devuelve una etiqueta para cada punto de datos según su Clúster
Y = kmeans.fit_predict(X)

In [ ]:
# graficando los clústers
plt.figure(figsize=(8,8))
plt.scatter(X[Y==0,0], X[Y==0,1], s=50, c="green", label="Cluster 1")
plt.scatter(X[Y==1,0], X[Y==1,1], s=50, c="red", label="Cluster 2")
plt.scatter(X[Y==2,0], X[Y==2,1], s=50, c="yellow", label="Cluster 3")
plt.scatter(X[Y==3,0], X[Y==3,1], s=50, c="violet", label="Cluster 4")
plt.scatter(X[Y==4,0], X[Y==4,1], s=50, c="blue", label="Cluster 5")

# graficando los centroides
plt.scatter(kmeans.cluster_centers_[:,0], kmeans.cluster_centers_[:,1], s=50, c="cyan", label="Centroides")

plt.title("Grupo de clientes")
plt.xlabel("Ingresos anuales")
plt.ylabel("Puntaje de gasto")
plt.legend()
plt.show()

In [ ]:
# agregando las etiquetas al dataframe
df["Cluster"] = kmeans.labels_ + 1
df.head()

Luego de seleccionar el mejor modelo, se procede a crear un sistema de predicción.

- ## Sistema de predicción

In [ ]:
def prediccion_fraude (data):
    # convirtiendo los datos en un array
    input_data_as_numpy_array= np.asarray(data)

    # reformando los datos
    input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

    # realizando predicción usando knn
    prediction = knn.predict(input_data_reshaped)

    # imprimiendo los resultados
    print(f'El resultado es {prediction[0]}')

    if (prediction[0]== 0):
      print("La transacción no es fraude")
    else:
      print("La transacción es fraude")

In [ ]:
datos_prediccion = (42230.09, 18, 0, 1, 5, 1, 0, 46, 20000000, 10000000, 1, 0.00, 1, 0.00, 4552.41)
prediccion_fraude(datos_prediccion)